In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from netCDF4 import Dataset

import matplotlib.image as mpimg
from matplotlib import cm
import matplotlib.animation as ani
from scipy.integrate import odeint
import pandas as pd
import copy

from sklearn.ensemble import RandomForestRegressor

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [2]:
isotopes = Dataset('climatology/xnapa_isotopes.nc', "a")
precipitation = Dataset('climatology/xnapa_precip.nc', "a")
surface_temperature = Dataset('climatology/xnapa_temp.nc', "a")

In [3]:
lat = isotopes.variables["latitude"][:].data[1:-1]
long = isotopes.variables["longitude"][:].data
t = isotopes.variables["t"][:].data

trash = [626, 0]

In [4]:
do18 = []
precip = []
temp = []
for i in range(len(t)):
    do18.append(isotopes.variables["dO18"][i][0].data[1:-1].reshape(71*96))
    precip.append(precipitation.variables["precip"][i][0].data[1:-1].reshape(71*96))
    temp.append(surface_temperature.variables["temp_1"][i][0].data[1:-1].reshape(71*96))
    
do18 = pd.DataFrame(np.delete(np.delete(np.asarray(do18), 0, 0), 626, 0)[:1000])
precip = pd.DataFrame(np.delete(np.delete(np.asarray(precip), 0, 0), 626, 0)[:1000])
temp = pd.DataFrame(np.delete(np.delete(np.asarray(temp), 0, 0), 626, 0)[:1000])

In [5]:
def rfr(temp, precip, do18, train_proportion=0.9, random=True):
    
    if random:
        random_idx = np.random.permutation(do18.index.values)
    
        temp = temp.reindex(random_idx)
        precip = precip.reindex(random_idx)
        do18 = do18.reindex(random_idx)
    
    #divide training and test data
    do18_train = do18[:int(np.floor(train_proportion*len(do18)))].reset_index(drop=True)
    do18_test = do18[int(np.floor(train_proportion*len(do18))):].reset_index(drop=True)

    temp_train = temp[:int(np.floor(train_proportion*len(temp)))].reset_index(drop=True)
    temp_test = temp[int(np.floor(train_proportion*len(temp))):].reset_index(drop=True)

    precip_train = precip[:int(np.floor(train_proportion*len(precip)))].reset_index(drop=True)
    precip_test = precip[int(np.floor(train_proportion*len(precip))):].reset_index(drop=True)
    
    
    X_train = np.append(temp_train, precip_train, axis=1)
    X_test = np.append(temp_test, precip_test, axis=1)

    Y_train = do18_train
    Y_test = pd.DataFrame(do18_test)
    
    Y = pd.DataFrame(np.append(Y_train, Y_test, axis=0))
    
    print("training...")
    #train model
    regressor = RandomForestRegressor().fit(X_train, Y_train)
    
    print("predicting...")
    #predict test data
    pred = pd.DataFrame(regressor.predict(X_test))
    
    
    #return pca and final data
    return pred, Y_test

In [ ]:
pred, Y_test = rfr(temp, precip, do18)

training...
